In [2]:
%load_ext autoreload
%autoreload 2
from ee_extract import *
import geemap

ee.Initialize()

# 截取栾城的数据
point = ee.Feature(ee.Geometry.Point([114 + 41/60, 37 + 53/60]), {'name': 'LuanCheng'})
filter = ee.Filter.calendarRange(2007, 2018, "year")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2
from ee_PML import *
from pkg_agg import *


In [3]:
# ERA5L
col_era5l = get_daily_ERA5L().filter(filter)
col_export_points(point, col, "data/栾城_ERA5L.csv")

In [4]:
# GLDAS
col_gldas = get_GLDAS().filter(filter)
col_gldas.limit(1)
# col_export_points(point, col_gldas, "data/栾城_GLDAS21.csv")

Name,Description
Albedo_inst,Albedo
AvgSurfT_inst,Average surface skin temperature
CanopInt_inst,Plant canopy surface water
ECanop_tavg,Canopy water evaporation
ESoil_tavg,Direct evaporation from bare soil
Evap_tavg,Evapotranspiration
LWdown_f_tavg,Downward long-wave radiation flux
Lwnet_tavg,Net long-wave radiation flux
PotEvap_tavg,Potential evaporation rate
Psurf_f_inst,Pressure


In [6]:
bands_mean = list(bands_GLDAS21.keys())[0:-1]
band_Tair = "Tair"

for year in range(2007, 2019):
    col = get_daily_GLDAS(year, bands_mean)
    col_export_points(point, col, f"栾城_GLDAS_{year}.csv")

## 截取数据


In [4]:
def tidy_pml(id):
    return ee.ImageCollection(id).filter(filter).map(cal_ETsum)

col_pml_v014 = tidy_pml("projects/pml_evapotranspiration/PML/OUTPUT/PML_V2_8day_v014")
col_pml_v017 = tidy_pml("CAS/IGSNRR/PML/V2_v017")
col_pml_china = tidy_pml("users/shaoyanghe96/PMLV2_China_daily")

col_pml_china.limit(3)

In [ ]:
%%time
col_export_points(point, col_pml_v014, "data/栾城_PMLV2_014")
col_export_points(point, col_pml_v017, "data/栾城_PMLV2_017")
col_export_points(point, col_pml_china, "data/栾城_PMLV2_China")


In [5]:
col_lai = ee.ImageCollection('MODIS/061/MOD15A2H') \
    .filter(filter) \
    .select(['Lai_500m'], ['LAI'])

vis_lai = {
    "min": 0, 
    "max": 20, 
    "palette": ["red", "blue", "green"]
}
# map = geemap.Map()
# map.addLayer(col_lai, vis_lai, "lai")
# map.add_colorbar(vis_lai, label="LAI", discrete=False);
# map

In [6]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.centerObject(point, 14)
Map.addLayer(col_lai, vis_lai, "lai")
Map.add_colorbar(vis_lai, label="LAI", discrete=False);
Map.addLayer(point, {'color': 'black', 'pointsize': 12}, 'point')
Map.addLayer(point.buffer(1000), {'color': 'red', 'PointSize': 12}, '1km', True, 0.4)
Map

Map(center=[37.88333333333333, 114.68333333333334], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# ?ee.batch.Export.table.toDrive
# "CSV" (default), "GeoJSON", "KML", "KMZ", "SHP", or "TFRecord"
# ee.batch.Export.table.toDrive(res, "temp", folder="gee", fileFormat="GeoJSON").start()